In [1]:
import os
%pwd

'e:\\mlops\\kidney_app\\research'

In [2]:
os.chdir("../")
%pwd

'e:\\mlops\\kidney_app'

In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/MutugiD/kidney_app.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="MutugiD"
os.environ["MLFLOW_TRACKING_PASSWORD"]="a804caf10c7510313d501aa76694ca2d26e38f1b"

In [4]:
#set MLFLOW_TRACKING_URI=https://dagshub.com/MutugiD/kidney_app.mlflow
#set MLFLOW_TRACKING_USERNAME=MutugiD
#set MLFLOW_TRACKING_PASSWORD=a804caf10c7510313d501aa76694ca2d26e38f1b

In [5]:
import tensorflow as tf
from kidneyDiseaseClassifier.constants import *
from kidneyDiseaseClassifier.utils.common import read_yaml, create_directories, save_json

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data:Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [7]:
class ConfigurationManager:

    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH) -> None:

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        
        evaluation_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/MutugiD/kidney_app.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return evaluation_config

In [8]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from mlflow.models import infer_signature
from urllib.parse import urlparse

In [9]:
class Evaluation:
    def __init__(self, config: EvaluationConfig) -> None:
        self.config = config

    def valid_generator(self):
        """
        Prepares data generators for training and validation.

        This method prepares data generators for training and validation using the specified parameters
        in the training configuration. It applies data augmentation techniques if enabled.
        """
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.validation_data = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.valid_generator()
        self.score = self.model.evaluate(self.validation_data)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store= urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [10]:
try:
    config = ConfigurationManager()
    evaluation_config = config.get_evaluation_config()
    evaluation = Evaluation(config=evaluation_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-08-07 01:50:47,191: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-07 01:50:47,201: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-07 01:50:47,203: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 13s 1s/step - loss: 17.9759 - accuracy: 0.5396
[2024-08-07 01:51:01,372: INFO: common: json file saved at: scores.json]


2024/08/07 01:51:02 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-08-07 01:51:04,069: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\admin\AppData\Local\Temp\tmprb6d2xe2\model\data\model\assets
[2024-08-07 01:51:04,567: INFO: builder_impl: Assets written to: C:\Users\admin\AppData\Local\Temp\tmprb6d2xe2\model\data\model\assets]


c:\Users\admin\anaconda3\envs\kidney\lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2024/08/07 01:51:47 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
